In [12]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import pandas as pd
import csv


# Import matplotlib libraries
# from matplotlib import pyplot as plt
# from matplotlib.collections import LineCollection
# import matplotlib.patches as patches

# import imageio
# from IPython.display import HTML, display


In [2]:
def img_crop (image):
  height=image.shape[0]
  width=image.shape[1]
  h_lower=int(height/2+ (0.45*height))
  h_upper=int(height/2- (0.38*height))
  w_left=int(width/2-(0.3*width))
  w_right=int(width/2+(0.2*width))
  #print(h_upper)
  image = image[h_upper:h_lower, w_left:w_right, :]

  return image

In [3]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=30):
    name = video_path.split("/")[-1].split(".")[0].lower()
    subname=name[6:]
    
  
  
    print(name)
    print(subname)
    if (name.find("_bhu") != -1) :
          save_path = os.path.join(save_dir, "Bhujangasana",subname)  
    elif (name.find("_padam") != -1) or (name.find("_padma") != -1):
          save_path = os.path.join(save_dir, "Padamasana",subname)
    elif (name.find("_shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana",subname)
    elif (name.find("_trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana",subname)
    elif (name.find("_vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana",subname)
    elif (name.find("_tad") != -1):
        save_path = os.path.join(save_dir, "Tadasana",subname)
    else:
        print(subname)
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            image=img_crop(frame)
            cv2.imwrite(f"{save_path}/{idx}.png", image)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("Yoga_Vid_Collected/train/*")
    save_dir = "dataset/train"
    

    for path in video_paths:
        save_frame(path, save_dir, gap=30)
    
    video_paths = glob("Yoga_Vid_Collected/test/*")
    save_dir = "dataset/test"
    
    for path in video_paths:
        save_frame(path, save_dir, gap=30)

train\abhay_bhuj
abhay_bhuj
dataset/train\Bhujangasana\abhay_bhuj
train\abhay_padam
abhay_padam
dataset/train\Padamasana\abhay_padam
train\abhay_shav
abhay_shav
dataset/train\Shavasana\abhay_shav
train\abhay_tada
abhay_tada
dataset/train\Tadasana\abhay_tada
train\abhay_trik
abhay_trik
dataset/train\Trikonasana\abhay_trik
train\abhay_vriksh
abhay_vriksh
dataset/train\Vrikshasana\abhay_vriksh
train\ameya_bhuj
ameya_bhuj
dataset/train\Bhujangasana\ameya_bhuj
train\ameya_padam
ameya_padam
dataset/train\Padamasana\ameya_padam
train\ameya_shav
ameya_shav
dataset/train\Shavasana\ameya_shav
train\ameya_tadasan
ameya_tadasan
dataset/train\Tadasana\ameya_tadasan
train\ameya_trikonasana
ameya_trikonasana
dataset/train\Trikonasana\ameya_trikonasana
train\ameya_vriksh
ameya_vriksh
dataset/train\Vrikshasana\ameya_vriksh
train\bhumi_bhuj
bhumi_bhuj
dataset/train\Bhujangasana\bhumi_bhuj
train\bhumi_padam
bhumi_padam
dataset/train\Padamasana\bhumi_padam
train\bhumi_shav
bhumi_shav
dataset/train\Shavasa

In [ ]:
def keypts_abs(keypoints):
  kpts_x = keypoints[0, 0, :, 0]
  kpts_y = keypoints[0, 0, :, 1]
  kpts_scores = keypoints[0, 0, :, 2]

  print(kpts_x.shape)
  kpts_absolute_xy = np.stack(
      [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)

  kpts_absolute_xy.shape

  keypoint_threshold=0.2
  kpts_above_thresh_absolute = kpts_absolute_xy[
      kpts_scores > keypoint_threshold, :]
  return kpts_above_thresh_absolute

In [44]:
data=[]
# KEYPOINT_DICT = [
#     'nose',
#     'left_eye',
#     'right_eye',
#     'left_ear',
#     'right_ear',
#     'left_shoulder',
#     'right_shoulder',
#     'left_elbow',
#     'right_elbow',
#     'left_wrist',
#     'right_wrist',
#     'left_hip',
#     'right_hip',
#     'left_knee',
#     'right_knee',
#     'left_ankle',
#     'right_ankle'
# ]
# for p in KEYPOINT_DICT:
        
#         data.append(p + "_x")
#         data.append(p + "_y")
#         data.append(p + "_vis")
# data.append( "_label")
data = pd.DataFrame(columns = data) # Empty dataset
print(data)
data.to_csv("dataset3.csv") # save the data as a csv file
# Load the input image.
def image_to_movenet(path,keypts_labels):
    
    if (path.find("Bhujangasana") != -1) :
          label="Bhujangasana"  
    elif (path.find("Padamasana") != -1):
          label="Padamasana"
    elif (path.find("Shavasana") != -1):
          label="Shavasana"
    elif (path.find("Trikonasana") != -1):
          label="Trikonasana"
    elif (path.find("Vrikshasana") != -1):
          label="Vrikshasana"
    elif (path.find("Tadasana") != -1):
          label="Tadasana"
        
    image_path = path
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)

    # Download the model from TF Hub.
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    movenet = model.signatures['serving_default']

    # Run model inference.
    outputs = movenet(image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints = outputs['output_0']
    kpts_x = keypoints[0, 0, :, 0]
    kpts_y = keypoints[0, 0, :, 1]
    kpts_scores = keypoints[0, 0, :, 2]
    
    keypoints_tup=(keypoints,label)
    keypts_labels.append(keypoints_tup)
    with open('dataset3.csv','w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['nose_x', 'nose_y', 'nose_vis', 'left_eye_x', 'left_eye_y', 'left_eye_vis', 'right_eye_x', 'right_eye_y', 'right_eye_vis', 'left_ear_x', 'left_ear_y', 'left_ear_vis', 'right_ear_x', 'right_ear_y', 'right_ear_vis', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_vis', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_vis', 'left_elbow_x', 'left_elbow_y', 'left_elbow_vis', 'right_elbow_x', 'right_elbow_y', 'right_elbow_vis', 'left_wrist_x', 'left_wrist_y', 'left_wrist_vis', 'right_wrist_x', 'right_wrist_y', 'right_wrist_vis', 'left_hip_x', 'left_hip_y', 'left_hip_vis', 'right_hip_x', 'right_hip_y', 'right_hip_vis', 'left_knee_x', 'left_knee_y', 'left_knee_vis', 'right_knee_x', 'right_knee_y', 'right_knee_vis', 'left_ankle_x', 'left_ankle_y', 'left_ankle_vis', 'right_ankle_x', 'right_ankle_y', 'right_ankle_vis', 'label'])
        for i in range(0,17):
            csv_out.writerows(kpts_x[i])
            csv_out.writerows(kpts_y[i])
            csv_out.writerows(kpts_scores[i])
        csv_out.writerows(label)  
#    
#     print(path)
#     print(keypts_labels)
    
    

Empty DataFrame
Columns: []
Index: []


In [45]:
import os
import time

path_lst=[]
keypts_labels_train=[]
keypts_labels_test=[]

class_list=os.listdir('dataset/train')
for name in class_list:
    for image in os.listdir(f"dataset/train/{name}"):
        for item in os.listdir(f"dataset/train/{name}/{image}"):
             add_tup=(os.path.abspath(f"dataset/train/{name}/{image}/{item}"),name)
             path_lst.append(os.path.abspath(f"dataset/train/{name}/{image}/{item}"))



path_lst = sorted(path_lst,key=os.path.getmtime)
for path in path_lst:
    image_to_movenet(path,keypts_labels_train)

# class_list=os.listdir('dataset/test')
# for name in class_list:
#     for image in os.listdir(f"dataset/test/{name}"):
#         for item in os.listdir(f"dataset/test/{name}/{image}"):
#              add_tup=(os.path.abspath(f"dataset/test/{name}/{image}/{item}"),name)
#              path_lst.append(os.path.abspath(f"dataset/test/{name}/{image}/{item}"))



# path_lst = sorted(path_lst,key=os.path.getmtime)
# for path in path_lst:
#     image_to_movenet(path,keypts_labels_test)    


<class 'tensorflow.python.framework.ops.EagerTensor'>


TypeError: Cannot iterate over a scalar tensor.